In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [3]:
def plot(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_' + metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])
    plt.show()

In [4]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True, shuffle_files =True)
train_examples, test_examples = dataset['train'], dataset['test']

In [5]:
train_examples

<_OptionsDataset shapes: ((None,), ()), types: (tf.int64, tf.int64)>

 The dataset `info` includes the encoder (a `tfds.features.text.SubwordTextEncoder`).

In [6]:
encoder = info.features['text'].encoder

In [7]:
print('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [8]:
sample_string = 'Hello World'

encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))

Encoded string is [4025, 222, 2307, 1829]
The original string: "Hello World"


In [9]:
for index in encoded_string:
  print('{} ----> {}'.format(index, encoder.decode([index])))

4025 ----> Hell
222 ----> o 
2307 ----> Wor
1829 ----> ld


## Prepare the training data

In [10]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [11]:
train_dataset = (train_examples
                 .shuffle(BUFFER_SIZE)
                 .padded_batch(BATCH_SIZE, padded_shapes=([None],[])))

test_dataset = (test_examples
                .padded_batch(BATCH_SIZE,  padded_shapes=([None],[])))

In [12]:
next(iter(train_dataset))

(<tf.Tensor: shape=(64, 1578), dtype=int64, numpy=
 array([[ 275,  126,  105, ...,    0,    0,    0],
        [  62,   18,   45, ...,    0,    0,    0],
        [  62,   77,  265, ...,    0,    0,    0],
        ...,
        [  12,   31,    4, ...,    0,    0,    0],
        [8002, 7968,  123, ...,    0,    0,    0],
        [4874, 8000, 4235, ...,    0,    0,    0]])>,
 <tf.Tensor: shape=(64,), dtype=int64, numpy=
 array([0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1])>)

## Creating model

Build a `tf.keras.Sequential` model and start with an embedding layer. An embedding layer stores one vector per word. When called, it converts the sequences of word indices to sequences of vectors. These vectors are trainable. After training (on enough data), words with similar meanings often have similar vectors.

This index-lookup is much more efficient than the equivalent operation of passing a one-hot encoded vector through a `tf.keras.layers.Dense` layer.

A recurrent neural network (RNN) processes sequence input by iterating through the elements. RNNs pass the outputs from one timestep to their input—and then to the next.

The `tf.keras.layers.Bidirectional` wrapper can also be used with an RNN layer. This propagates the input forward and backwards through the RNN layer and then concatenates the output. This helps the RNN to learn long range dependencies.

## Stack two or more LSTM layers

Keras recurrent layers have two available modes that are controlled by the `return_sequences` constructor argument:

* Return either the full sequences of successive outputs for each timestep (a 3D tensor of shape `(batch_size, timesteps, output_features)`).
* Return only the last output for each input sequence (a 2D tensor of shape (batch_size, output_features)).

In [13]:
"""
def build_model(input_shape):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_shape, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
    ])
    
    return model
"""
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])



In [14]:
#model = build_model(encoder.vocab_size)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 635,329
Trainable params: 635,329
Non-trainable params: 0
__________________________________________________

In [15]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [16]:
history = model.fit(train_dataset, epochs=1,
                    validation_data=test_dataset, 
                    validation_steps=30)

      2/Unknown - 26s 13s/step - loss: 0.6948 - accuracy: 0.4219

KeyboardInterrupt: 

In [17]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

     31/Unknown - 34s 1s/step - loss: 0.6931 - accuracy: 0.5120

KeyboardInterrupt: 

In [18]:
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

In [27]:
def sample_predict(sample_pred_text, pad):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)
  #print(encoded_sample_pred_text)

  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  #print(encoded_sample_pred_text)
  #predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  #return (predictions)

### Predict on a sample text without padding

In [28]:
sample_pred_text = ('The movie was not good. The animation and the graphics '
                    'were terrible. I would not recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=False)
#print(predictions)

[19, 27, 18, 33, 248, 3, 19, 1847, 5, 1, 5172, 8, 85, 1751, 3, 12, 70, 33, 505, 14, 65, 7975]
tf.Tensor(
[1.900e+01 2.700e+01 1.800e+01 3.300e+01 2.480e+02 3.000e+00 1.900e+01
 1.847e+03 5.000e+00 1.000e+00 5.172e+03 8.000e+00 8.500e+01 1.751e+03
 3.000e+00 1.200e+01 7.000e+01 3.300e+01 5.050e+02 1.400e+01 6.500e+01
 7.975e+03], shape=(22,), dtype=float32)


### Predict on a sample text with padding

In [24]:
# predict on a sample text with padding

sample_pred_text = ('The movie was not good. The animation and the graphics '
                    'were terrible. I would not recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=True)
#print(predictions)

[19, 27, 18, 33, 248, 3, 19, 1847, 5, 1, 5172, 8, 85, 1751, 3, 12, 70, 33, 505, 14, 65, 7975]


In [25]:
plot(history, 'accuracy')

NameError: name 'history' is not defined

In [26]:
plot(history, 'loss')

NameError: name 'history' is not defined